# Machine Learning analysis of data about Competency Assessment of Students from HE Biotech Academic Programs

**Index**
>
> 1. [Data loading and preparation for Machine Learning for Stratified 10-fold Cross-Validation.](#1-data-loading-and-preparation-for-machine-learning-for-stratified-10-fold-cross-validation)
>
> 2. [Random Forest definition, training, and validation.](#2-random-forest-definition-training-and-validation)
>
> 3. [Deep Learning neural network for one-class classification with Pytorch, definition, training, and validation.](#3-deep-learning-neural-network-for-one-class-classification-with-pytorch-definition-training-and-validation)
>
> 4. [Statistical tests for comparing RandomForest and Deep Leaning model across all folds](#4-statistical-tests-for-comparing-randomforest-and-deep-leaning-model-across-all-folds)
>
> 5. [Interpretability of the RandomForest using SHAP values](#5-interpretability-of-the-randomforest-using-shap-values)
>
> 6. [PCA and FAMD analysis](#6-factor-analysis-of-mixed-data)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score, RocCurveDisplay
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix, accuracy_score, \
    precision_score, recall_score, f1_score, classification_report, precision_score, \
    precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import os
import seaborn as sns

## 1. Data loading and preparation for Machine Learning for Stratified 10-fold Cross-Validation.

In [ ]:
# Loading dataframe and showing some information
bio_df = pd.read_csv('sourcecode/EICData/Bio_students_v5.6.csv')
bio_df.drop(columns=['Unnamed: 0'], inplace=True)
# Remove "ID" column
bio_df.drop('ID', axis=1, inplace=True) 
bio_df['last_col'] = bio_df['Assigned']
bio_df.drop(columns=['Assigned', 'isRegular'], inplace=True)
bio_df.rename(columns={'last_col':'Assigned'}, inplace=True)
# There were an empty string category of the variable Modality
bio_df['Modality'] = bio_df['Modality'].replace(r'^\s*$', 'Unknown', regex=True)
# Remove records with any NaN 
bio_df.dropna(inplace=True)
bio_df.shape

In [ ]:
# Excluding also boolean variables include=['object']
columnas_categorical = bio_df.select_dtypes(exclude=['int64', 'float64']).columns.tolist() 
# Excluding also boolean variables
columnas_numerical = bio_df.select_dtypes(include=['int64', 'float64']).columns.tolist() 

In [ ]:
device = 0 if torch.cuda.is_available() else 'cpu'
batch_size = 32

In [ ]:
# Dataset preparation
# Remove class var if it is in the categorical list
if 'Assigned' in (columnas_categorical):
    columnas_categorical.remove('Assigned')
# Remove class var if it is in the numeric list
if 'Assigned' in (columnas_numerical):
    columnas_numerical.remove('Assigned')

# Encoding categorical columns
df_clean_shuffled = bio_df.sample(frac=1)
y = df_clean_shuffled['Assigned'].astype('int')

# Encode full dataset first
# We have avoid dropping the first category, because RandomForest is better if having all categories explicitly
full_preprocessor = ColumnTransformer([
    ('num', MinMaxScaler(), columnas_numerical),
    ('cat', OneHotEncoder(handle_unknown='ignore', #drop='None', 
                          sparse_output=False), 
            columnas_categorical)
])
X_encoded = full_preprocessor.fit_transform(df_clean_shuffled)

# Ensure no value is greater than 1.0
X_encoded = np.clip(X_encoded, 0, 1)

# Get feature names after encodings
encoded_features = []
for col in full_preprocessor.transformers_:
    if col[0] == 'num':
        encoded_features.extend(col[2])  # numeric columns
    elif col[0] == 'cat':
        # Get one-hot encoded names
        encoder = col[1]
        # Comment if not using drop='First'
        encoded_features.extend([f"{col[2][i]}_{cat}" 
                               for i in range(len(col[2])) 
                               for cat in encoder.categories_[i]])
        # Uncomment if using drop='First'
        # categories = encoder.categories_
        # for i, feature in enumerate(col[2]):
        #     # Skip the first category if drop='first'
        #     for cat in categories[i][1:]:
        #         encoded_features.append(f"{feature}_{cat}")

# Convert back to DataFrame with proper column names
X_encoded = pd.DataFrame(X_encoded, columns=encoded_features)
X_encoded.shape

In [ ]:
def cross_val_splitter(X_encoded, y, n_splits=10):
    #  Modficar para realizar entrenamiento del modelo con los datos de X_data
    Train = X_encoded    
    labels = y
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    cross_val_data = skf.split(Train, labels)
    return cross_val_data

In [ ]:
# Using the same cross-validation splits and codified data for both algorithms
cross_val_idxs = cross_val_splitter(X_encoded, y, n_splits = 10)

# Create a list to store each fold's data
cross_val_folds = []

# Loop through each fold and store the indices in a dictionary
for fold, (train, test) in enumerate(cross_val_idxs):
    fold_data = {
        'fold': fold,
        'train': list(train.astype('int')),
        'test': list(test.astype('int'))
    }
    cross_val_folds.append(fold_data)

# for fold in cross_val_folds:
#     print(f'Train: {cross_val_folds[0]['train']}')
#     print(f'Test {cross_val_folds[0]['test']}')


## 2. Random Forest definition, training, and validation

In [ ]:
# Random Forest Setup

rf_biotech = RandomForestClassifier(
    n_estimators=100,  # Default number of trees
    random_state=42,
    class_weight='balanced', # It balance the weight for each class accoridng to the imbalance ratio
    verbose=False,
    n_jobs=8
)

In [ ]:
def train_validate_cross_val_RandomForest(cross_val_idxs, X_encoded, y, rf_biotech):
    print('Stratified 10-Fold Cross-Validation with RandomForest')

    # Initialize True Positive Rate and Area Under Curve
    os.makedirs("fold_plots", exist_ok=True)
    tprs, aucs = [], []
    mean_fpr = np.linspace(0, 1, 100)

    # Métricas de estudio
    metrics = {
        'Iteration': [],
        'AUC': [],
        'Precision_Class0': [],
        'Precision_Class1': [],
        'Recall_Class0': [],
        'Recall_Class1': [],
        'F1_Class0': [],
        'F1_Class1': [],
        'ConfusionMatrix': []
    }

    # Plot ROC Curve for every Cross Validation Split
    plt.figure(figsize=(8, 6))
    fig, ax = plt.subplots()
    # Use manually stratified folds to explore the entire dataset
    # fold_size = int(len(X_encoded.index)/n_splits)
    # for index in range(n_splits):
    for fold in cross_val_idxs:
        # Split the data
        index = fold['fold']
        train_idx = fold['train']
        test_idx = fold['test']
        print(f'\nFold {index}, Train size {len(train_idx)}, Test size {len(test_idx)}')
        X_train_fold, X_val_fold = X_encoded[train_idx], X_encoded[test_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx].values, y.iloc[test_idx].values
        # X_train, X_test = X_encoded.iloc[train_idx], X_encoded.iloc[test_idx]
        # y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        rf_biotech.fit(X_train_fold, y_train_fold) # This statement retrains the model from scratch
        # model_loaded.fit(X_train, y_train) # I am affraid this statement is using the model pretrained with train data
        # Calcular la Curva AUC
        plot = RocCurveDisplay.from_estimator(
            rf_biotech, X_val_fold, y_val_fold,
            # name="Fold {}".format(index),
            ax=ax,
        )

        # Then, update the label with formatted AUC
        plot.line_.set_label("Fold {} (AUC = {:.4f})".format(index, plot.roc_auc))

        # Interpolate TPR for averaging
        interp_tpr = np.interp(mean_fpr, plot.fpr, plot.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(plot.roc_auc)

        # Predecir los datos de entrenamiento
        y_pred = rf_biotech.predict(X_val_fold)

        # Guardar métricas en un diccionario
        metrics['Iteration'].append(index + 1)
        metrics['AUC'].append(plot.roc_auc)
        metrics['ConfusionMatrix'].append(confusion_matrix(y_val_fold, y_pred))
        for cls in [0, 1]:
            metrics[f'Precision_Class{cls}'].append(precision_score(y_val_fold, y_pred, pos_label=cls, zero_division=0))
            metrics[f'Recall_Class{cls}'].append(recall_score(y_val_fold, y_pred, pos_label=cls, zero_division=0))
            metrics[f'F1_Class{cls}'].append(f1_score(y_val_fold, y_pred, pos_label=cls, zero_division=0))
    
    ax.set(
        xlim=[-0.05, 1.05],
        ylim=[-0.05, 1.05],
        title="ROC Curves Across Folds",
    )
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig("fold_plots/roc_cv_randomForest_kfold.pdf")

    # Convert to DataFrame to save it and analyze it
    metrics_df = pd.DataFrame(metrics)
    metrics_df.to_csv('fold_plots/RandomForest_Metric_Summary.csv', index=False)
    return metrics_df

In [ ]:
rf_metrics_summary = train_validate_cross_val_RandomForest(cross_val_folds, X_encoded, y, rf_biotech)

## 3. Deep Learning neural network for one-class classification with Pytorch, definition, training, and validation.

In [ ]:
class OneClassNN(nn.Module):
    def __init__(self, input_dim):
        super(OneClassNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.sigmoid(self.fc6(x)) # Ensure this line is present
        return x


In [ ]:
# Function to train, validate, and test the model
def train_validate_test_model(cross_val_idxs, X_encoded, y, batch_size=batch_size, epochs=100, learning_rate=0.0001, patience=10):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")       
        
    # Lists to store results
    all_y_true = []
    all_y_pred = []
    fold_auc_scores = []
    fold_metrics = []
    best_y_pred = []

    for index in cross_val_idxs:
        fold = index['fold']
        train_idx = index['train']
        test_idx = index['test']        
        # Split the data
        print(f'\nFold {fold}, Train size{len(train_idx)}, Test size {len(test_idx)}')
        X_train_fold, X_val_fold = X_encoded[train_idx], X_encoded[test_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx].values, y.iloc[test_idx].values

        X_train_tensor = torch.tensor(X_train_fold, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train_fold, dtype=torch.float32).view(-1, 1).to(device)

        X_val_tensor = torch.tensor(X_val_fold, dtype=torch.float32).to(device)
        y_val_tensor = torch.tensor(y_val_fold, dtype=torch.float32).view(-1, 1).to(device)

        # Create data loaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        # Initialize model, loss function, and optimizer
        model = OneClassNN(X_encoded.shape[1])
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        model.to(device)

        # Early stopping parameters
        best_val_AUC = 0
        patience_counter = 0

        # Lists to store loss values
        train_losses = []
        val_losses = []

        # Training loop
        for epoch in range(epochs):
            model.train()
            train_loss = 0
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            train_loss /= len(train_loader)
            train_losses.append(train_loss)
        
            # Validation loop
            model.eval()
            val_loss = 0
            y_true = []
            y_pred = []
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch = X_batch.to(device)
                    y_batch = y_batch.to(device)
                    outputs = model(X_batch)
                    y_true.extend(y_batch.cpu().numpy())
                    y_pred.extend(outputs.cpu().numpy())
                    loss = criterion(outputs, y_batch)
                    val_loss += loss.item()

            val_loss /= len(val_loader)
            val_losses.append(val_loss)
            epoch_auc = roc_auc_score(y_true, y_pred)

            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val AUC: {epoch_auc:.4f}")
            
            # Early stopping check for validation_AUC
            if epoch_auc >= best_val_AUC:
                best_val_AUC = epoch_auc
                best_y_pred = y_pred
                patience_counter = 0
                # Save the best model
                torch.save(model.state_dict(), f'best_model_epoch{epoch}.pth')
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping triggered at epoch {epoch} with AUC = {best_val_AUC:.4f}")
                    break
            
        # Store preformance metrics
        all_y_true.extend(y_true)
        all_y_pred.extend(best_y_pred)
        fold_auc_scores.append(best_val_AUC)

        # Adding standard metrics
        y_pred_binary = (np.array(best_y_pred) >= 0.5).astype(int)
        y_true_binary = np.array(y_true).astype(int)

        precision = precision_score(y_true_binary, y_pred_binary, average=None)
        recall = recall_score(y_true_binary, y_pred_binary, average=None)
        f1 = f1_score(y_true_binary, y_pred_binary, average=None)

        precision0, precision1 = precision
        recall0, recall1 = recall
        f1_0, f1_1 = f1
        
        conf_matrix = confusion_matrix(y_true_binary, y_pred_binary, labels=[0,1])
        fold_metrics.append({
            'Fold': fold,
            'AUC': best_val_AUC,
            'best_y_pred': best_y_pred,
            'y_true': y_true,
            'Precision1': precision1,
            'Recall1': recall1,
            'F1_1': f1_1,
            'Precision0': precision0,
            'Recall0': recall0,
            'F1_0': f1_0,
            'ConfusionMatrixTN': conf_matrix[0][0],
            'ConfusionMatrixFP': conf_matrix[0][1],
            'ConfusionMatrixFN': conf_matrix[1][0],
            'ConfusionMatrixTP': conf_matrix[1][1]
        })

        # # Save metrics to a Pandas Dataframe
        # metrics = pd.DataFrame({
        #     'Fold': list(range(1, 11)),
        #     'AUC': fold_auc_scores
        # }).to_csv("fold_auc_scores.csv", index=False)

        # Plot training and validation loss
        plt.figure()
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title(f'Training and Validation Loss Fold {fold}')
        plt.legend()
        plt.savefig(f'fold_plots/training_validation_loss_fold_{fold}.pdf')
        # plt.show()

        print(classification_report(y_true_binary, y_pred_binary))


    # Final metrics
    overall_auc = roc_auc_score(all_y_true, all_y_pred)
    print(f"\nAverage AUC across folds: {np.mean(fold_auc_scores):.4f}")
    print(f"Final ROC AUC: {overall_auc:.4f}")

    # Plot final ROC curve
    
    plt.figure(figsize=(8, 6))
    for fold_data in fold_metrics:
        fpr, tpr, _ = roc_curve(fold_data['y_true'], fold_data['best_y_pred'])
        plt.plot(fpr, tpr, label=f"Fold {fold_data['Fold']} AUC = {fold_data['AUC']:.4f}")
    # plt.plot([0, 1], [0, 1], 'k--', label='Chance')
    plt.xlabel("False Positive Rate (Positive label: 1)")
    plt.ylabel("True Positive Rate (Positive label: 1)")
    plt.title("ROC Curves Across Folds")
    plt.legend(loc="lower right")
    plt.grid(True)
    plt.savefig("fold_plots/final_roc_curves.pdf")
    plt.show()

    metrics_df = pd.DataFrame(fold_metrics)
    metrics_df.drop(columns=['best_y_pred', 'y_true'], inplace=True)
    metrics_df.to_csv("fold_plots/OCC_Metric_Summary.csv", index=False)
    print("\nPer-fold metrics saved to 'OCC_Metric_Summary.csv'")

    return model, fold_metrics, metrics_df

In [ ]:
# Train, validate, and test the model
model, main_metrics_result, standard_metrics = train_validate_test_model(cross_val_folds, X_encoded, y)

### 4. Statistical tests for comparing RandomForest and Deep Leaning model across all folds

#### 4.1 Statisitical tests

In [ ]:
# View metrics
rf_metrics_summary

In [ ]:
# View metrics
standard_metrics

In [ ]:
from scipy.stats import ttest_rel, wilcoxon

# Example AUC scores from 10-fold cross-validation for two algorithms
auc_algorithm_rf = rf_metrics_summary['AUC'].to_numpy()
auc_algorithm_occ = standard_metrics['AUC'].to_numpy()

# Perform paired t-test
t_stat, p_value_ttest = ttest_rel(auc_algorithm_rf, auc_algorithm_occ)

# Perform Wilcoxon signed-rank test
w_stat, p_value_wilcoxon = wilcoxon(auc_algorithm_rf, auc_algorithm_occ)

# Print results
print(f"Paired t-test: t-statistic = {t_stat:.4f}, p-value = {p_value_ttest:.4f}")
print(f"Wilcoxon signed-rank test: w-statistic = {w_stat:.4f}, p-value = {p_value_wilcoxon:.4f}")

# Interpretation
alpha = 0.05
if p_value_ttest < alpha:
    print("Paired t-test: There is a significant difference between the AUC scores of the two algorithms.")
else:
    print("Paired t-test: There is no significant difference between the AUC scores of the two algorithms.")

if p_value_wilcoxon < alpha:
    print("Wilcoxon signed-rank test: There is a significant difference between the AUC scores of the two algorithms.")
else:
    print("Wilcoxon signed-rank test: There is no significant difference between the AUC scores of the two algorithms.")



#### 4.2 Mean AUC plots

In [ ]:

# Calculate statistics for RandomForest
rf_mean = np.mean(rf_metrics_summary['AUC'])
rf_median = np.median(rf_metrics_summary['AUC'])
rf_min = np.min(rf_metrics_summary['AUC'])
rf_max = np.max(rf_metrics_summary['AUC'])
rf_q1 = np.percentile(rf_metrics_summary['AUC'], 25)
rf_q3 = np.percentile(rf_metrics_summary['AUC'], 75)

# Calculate statistics for Deep Neural Network for OCC
occ_mean = np.mean(standard_metrics['AUC'])
occ_median = np.median(standard_metrics['AUC'])
occ_min = np.min(standard_metrics['AUC'])
occ_max = np.max(standard_metrics['AUC'])
occ_q1 = np.percentile(standard_metrics['AUC'], 25)
occ_q3 = np.percentile(standard_metrics['AUC'], 75)

data = {
    'Fold': list(range(1, 11)),
    'RandomForest': auc_algorithm_rf,
    'Deep Neural Network for OCC': auc_algorithm_occ
}

df = pd.DataFrame(data)

# Plotting the boxplots
plt.figure(figsize=(8, 6))
boxplot = df[['RandomForest', 'Deep Neural Network for OCC']].boxplot()

# Annotate statistics for RandomForest
plt.text(1.1, rf_mean, f'Mean: {rf_mean:.4f}', horizontalalignment='center', color='blue')
plt.text(1.1, rf_median, f'Median: {rf_median:.4f}', horizontalalignment='center', color='green')
plt.text(1.1, rf_min, f'Min: {rf_min:.4f}', horizontalalignment='center', color='red')
plt.text(1.1, rf_max, f'Max: {rf_max:.4f}', horizontalalignment='center', color='red')
plt.text(1.1, rf_q1, f'Q1: {rf_q1:.4f}', horizontalalignment='center', color='purple')
plt.text(1.1, rf_q3, f'Q3: {rf_q3:.4f}', horizontalalignment='center', color='purple')

# Annotate statistics for Deep Neural Network for OCC
plt.text(2.1, occ_mean, f'Mean: {occ_mean:.4f}', horizontalalignment='center', color='blue')
plt.text(2.1, occ_median, f'Median: {occ_median:.4f}', horizontalalignment='center', color='green')
plt.text(2.1, occ_min, f'Min: {occ_min:.4f}', horizontalalignment='center', color='red')
plt.text(2.1, occ_max, f'Max: {occ_max:.4f}', horizontalalignment='center', color='red')
plt.text(2.1, occ_q1, f'Q1: {occ_q1:.4f}', horizontalalignment='center', color='purple')
plt.text(2.1, occ_q3, f'Q3: {occ_q3:.4f}', horizontalalignment='center', color='purple')

plt.title('AUC Scores Comparison Across 10 Folds')
plt.ylabel('AUC Score')
plt.xlabel('Algorithm')
plt.grid(True)
plt.show()



## 5. Interpretability of the RandomForest using SHAP values 

### 5.1 Train again RandomForest with the entire dataset to compute SHAP values

In [ ]:
full_RF_model = rf_biotech.fit(X_encoded, y)

### 5.2 Plot graphbars with each varaible contribution to the decision of the classifier.

In [ ]:
# Obtain the feature importance
importances = full_RF_model.feature_importances_

# Create a DataFrame to store the importance 
feature_importance_df = pd.DataFrame({
    'Feature': X_encoded.columns,
    'Contribution': importances
})

# Create mapping to original columns to retrieve the original encoded features
importance_mapping = {}
for orig_col in columnas_categorical:
    importance_mapping[orig_col] = feature_importance_df[
        feature_importance_df['Feature'].str.startswith(orig_col)
    ]['Contribution'].sum()

for num_col in columnas_numerical:
    importance_mapping[num_col] = feature_importance_df[
        feature_importance_df['Feature'] == num_col
    ]['Contribution'].values[0]

# Create final importance dataframe
final_importance = pd.DataFrame(
    importance_mapping.items(),
    columns=['Feature', 'Contribution']
).sort_values('Contribution', ascending=False)

print("Feature Importance (Original Columns):")
print(final_importance.head())

In [ ]:
plt.figure(figsize=(12, 8))
number = 50
sns.barplot(x='Contribution', y='Feature', data=final_importance.head(number))

for index, value in enumerate(final_importance.head(number)['Contribution']):
    plt.text(value, index, f'{value:.4f}', va='center')

plt.title(f'Top Feature Importances')
plt.tight_layout()
plt.show()

### 5.3 SHAP values and plots

In [ ]:
import shap

In [ ]:
# Use 1% of the training data for SHAP explanation
sample_size = int(0.01 * len(X_encoded))
X_sample = X_encoded.sample(n=sample_size, random_state=42)

# Create a SHAP TreeExplainer for the Random Forest
explainer = shap.TreeExplainer(full_RF_model)

# Compute SHAP values for the 1% sample
shap_values = explainer.shap_values(X_sample)

# Plot SHAP summary plot for class 1 (positive class)
shap.summary_plot(shap_values[1], X_sample, plot_type="bar")  # use plot_type="dot" for the classic beeswarm

In [ ]:
# Plot SHAP values of the 1% of the dataset and the RandomForest turning back to the initial variables before OneHotEncoding.
# Sample 1% of the encoded dataset
sample_size = int(0.01 * len(X_encoded))
X_sample = X_encoded.sample(n=sample_size, random_state=42)

# Compute SHAP values
explainer = shap.TreeExplainer(full_RF_model)
shap_values = explainer.shap_values(X_sample)[1] # For class 1

# Create a mapping from encoded columns to original features
encoded_columns = X_encoded.columns
original_columns = bio_df.columns

# Example: {'gender_Male': 'gender', 'gender_Female': 'gender', 'age': 'age'}
encoded_to_original = {}
for col in original_columns:
    if bio_df[col].dtype == 'object' or bio_df[col].dtype.name == 'category':
        for val in bio_df[col].unique():
            encoded_col = f"{col}_{val}"
            if encoded_col in encoded_columns:
                encoded_to_original[encoded_col] = col
    else:
        if col in encoded_columns:
            encoded_to_original[col] = col

# Aggregate SHAP values by original feature
aggregated_shap = pd.DataFrame(index=X_sample.index)
for original_feature in original_columns:
    matching_encoded = [col for col, orig in encoded_to_original.items() if orig == original_feature]
    aggregated_shap[original_feature] = shap_values[:, [encoded_columns.get_loc(col) for col in matching_encoded]].sum(axis=1)

# Plot the summary
shap.summary_plot(aggregated_shap.values, features=aggregated_shap, feature_names=aggregated_shap.columns, plot_type="bar")


## 6. Factor Analysis of Mixed Data

### 6.1 FAMD with the initial dataset with mixed data

In [ ]:
import prince

# Sample 50% of the dataset and separate independent and dependent variables.
sample_df = bio_df.sample(frac=0.5)
X = sample_df.drop(columns=['Assigned'])
y = sample_df['Assigned']

# Apply FAMD with 5 components to determine the ideal number of components using the elbow method in the scree plot
# or the Cumulative Explaned Variance between 70% and 90%
# famd = prince.FAMD(n_components=5, random_state=42)
# Apply FAMD with the ideal number of components
famd = prince.FAMD(n_components=3, random_state=42)
famd = famd.fit(X)
X_famd = famd.transform(X)

In [ ]:
famd.scree_plot()

In [ ]:

# Compute explained inertia manually
U, s, V = np.linalg.svd(famd.row_coordinates(X), full_matrices=False)
explained_inertia = (s ** 2) / np.sum(s ** 2)

# Print explained inertia for each component
for i, inertia in enumerate(explained_inertia, start=1):
    print(f"Component {i}: {inertia:.4f} (Explained Variance)")

# Cumulative explained inertia
cumulative_inertia = np.cumsum(explained_inertia) * 100
print("\nCumulative Explained Variance:")
for i, cum_inertia in enumerate(cumulative_inertia, start=1):
    print(f"Component {i}: {cum_inertia:.4f}")

plt.figure(figsize=(10, 6))
# Plot cummulative Variance
plt.plot(range(1, len(cumulative_inertia) + 1), cumulative_inertia, marker='o', 
         linestyle='--', color='b', label='Cummulative Explained Variance')
# Plot individual Variance
plt.plot(range(1, len(explained_inertia) + 1), explained_inertia * 100, marker='^', 
         linestyle='--', color='r', label='Explained Variance')
# Annotate cummulative Variance
for idx, item in enumerate(cumulative_inertia):
    plt.text(idx+1.3, item, f'{item:.2f}', horizontalalignment='right', color='blue')
# Annotate Explained Variance
for idx, variance in enumerate(explained_inertia):
    plt.text(idx+1, variance * 100, f'{(variance*100):.2f}', horizontalalignment='left', verticalalignment='top', color='r')
plt.title('Explained Variance & Cumulative Explained Variance by FAMD Components')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance')
plt.axhline(y=70, color='black', linestyle='--', label='70% Variance')
plt.axhline(y=90, color='black', linestyle='--', label='90% Variance')
plt.xticks(range(1, len(cumulative_inertia) + 1))
plt.yticks(range(0, 110, 10))
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# Get coordinates (loadings) for categorical variable levels
categorical_coords = famd.column_coordinates_
# categorical_coords.index.name = 'variable'
categorical_coords.rename(columns={0:'Component1', 1:'Component2', 2:'Component3', 3:'Component4', 4:'Component5'}, 
                          inplace=True)

# Compute correlations for numerical variables
numerical_corrs = []
for i in range(X_famd.shape[1]):
    component = X_famd.iloc[:, i]
    corr = X[columnas_numerical].corrwith(component)
    numerical_corrs.append(corr)

# Convert to DataFrame with same shape as categorical_coords
numerical_corrs_df = pd.concat(numerical_corrs, axis=1)
numerical_corrs_df.columns = categorical_coords.columns  # Match FAMD component names
numerical_corrs_df.index.name = 'variable'

# Combine into a single DataFrame
# combined_contributions = categorical_coords.merge(how='left', right=numerical_corrs_df, on='variable')
combined_contributions = categorical_coords
combined_contributions.update(numerical_corrs_df)

# # Optional: sort for better visualization
# combined_contributions = combined_contributions.sort_index()

# Display result
print(combined_contributions)



In [ ]:
# Square the correlations to get contribution approximation
contrib = combined_contributions ** 2
contrib_norm_df = contrib.div(contrib.sum(axis=0), axis=1) * 100  # Normalize to get % contribution
# top_contrib = contrib_norm_df.apply(lambda x: x.sort_values(ascending=False).head(29))
for col in contrib_norm_df.columns:
    top_contrib = contrib_norm_df.sort_values(ascending=False, by=col)
    print(top_contrib[col])

In [ ]:
# Reset index so 'variable' becomes a column
df_melted = contrib_norm_df.reset_index().melt(id_vars='variable', 
                                                var_name='Component', 
                                                value_name='Contribution')
df_melted.rename(columns={'variable': 'Variable'}, inplace=True)

# Sort variables by average contribution across components (optional, for visual clarity)
df_melted['Variable'] = pd.Categorical(
    df_melted['Variable'],
    categories=(contrib_norm_df.mean(axis=1)
                .sort_values(ascending=False)
                .index.tolist()),
    ordered=True
)

# Set the color palette for the components
palette = sns.color_palette(n_colors=contrib_norm_df.shape[1])

# Create the barplot
plt.figure(figsize=(18, 6))
sns.barplot(
    data=df_melted,
    x='Variable',
    y='Contribution',
    hue='Component',
    palette=palette,
)

# Plot formatting
plt.xticks(rotation=90, fontsize=16)
plt.yticks(fontsize=16)
plt.ylabel('Contribution (%)', fontsize=18)
plt.xlabel('Variable', fontsize=18)
plt.title('Variable Contributions to FAMD Components', fontsize=20)
plt.legend(title='Component', fontsize=18)
plt.tight_layout()
plt.show()


In [ ]:
# Project individuals and variables
row_coords = famd.row_coordinates(X)  # individuals
col_coords = famd.column_correlations(X)  # variables

# Plot
fig, ax = plt.subplots(figsize=(10, 8))

# Plot individuals (optional: add color by y if you want classes)
ax.scatter(row_coords[0], row_coords[1], alpha=0.5, label='Individuals')

# Plot variable vectors
for i in range(col_coords.shape[0]):
    ax.arrow(0, 0, 
             col_coords.iloc[i, 0], col_coords.iloc[i, 1], 
             color='red', alpha=0.7, head_width=0.02)
    ax.text(col_coords.iloc[i, 0]*1.1, col_coords.iloc[i, 1]*1.1, 
            col_coords.index[i], color='red', ha='center', va='center')

ax.axhline(0, linestyle='--', color='grey')
ax.axvline(0, linestyle='--', color='grey')
ax.set_xlabel('FAMD Component 1')
ax.set_ylabel('FAMD Component 2')
ax.set_title('FAMD Biplot: Variables and Individuals')
plt.grid(True)
plt.tight_layout()
plt.show()

### 6.2 PCA with the encoded dataset

In [ ]:
# import prince
from sklearn.decomposition import PCA

# Select randomly 60% of the dataset
X_sample_full = X_encoded.copy()
X_sample_full['Assigned'] = y
X_sample = X_sample_full.sample(frac=1.0)
y_sample = X_sample['Assigned']
X_sample.drop(columns=['Assigned'], inplace=True)
del X_sample_full


pca = PCA(n_components=30)
X_pca = pca.fit_transform(X_sample)

# Scree plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, 'o-', color='blue')
plt.title('Scree Plot (PCA)')
plt.xlabel('Component Number')
plt.ylabel('Explained Variance Ratio')
plt.grid(True)
plt.show()

# Individual factor map
individuals = pd.DataFrame(X_pca, columns=[f'F{i+1}' for i in range(pca.n_components)])
individuals['DependentBoolean'] = y_sample.values

plt.figure(figsize=(10, 8))
sns.scatterplot(data=individuals, x='F1', y='F2', hue='DependentBoolean', palette='viridis')
plt.title('Individual Factor Map (F1 vs F2)')
plt.xlabel('F1')
plt.ylabel('F2')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:

# Calculate cumulative explained variance
cumulative_variance = pca.explained_variance_ratio_.cumsum()

# Plot
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--', color='b')
plt.title('Cumulative Explained Variance by PCA Components')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.axhline(y=0.7, color='r', linestyle='--', label='70% Variance')
plt.axhline(y=0.9, color='g', linestyle='--', label='90% Variance')
plt.grid(True)
plt.legend()
plt.show()
